In [30]:
import numpy as np
from tensorflow.keras import utils, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, StratifiedKFold
#from keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [31]:
train_directory = "/home/adarsh/Jupyter_Notebook/American Sign Lang/ASL Data/asl_alphabet_train/asl_alphabet_train"

In [32]:
classes = 29
batch = 256
epochs = 50
lnr = 0.0001

In [33]:
gen = ImageDataGenerator(rescale=1./255, validation_split=0.1) 
train = gen.flow_from_directory("/home/adarsh/Jupyter_Notebook/American Sign Lang/ASL Data/asl_alphabet_train/asl_alphabet_train", 
                                                            target_size=(64, 64), subset="training")
val = gen.flow_from_directory("/home/adarsh/Jupyter_Notebook/American Sign Lang/ASL Data/asl_alphabet_train/asl_alphabet_train", 
                                                            target_size=(64, 64), subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [35]:
adam = Adam(learning_rate=lnr)
model = Sequential()
model.add(VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(29, activation='softmax'))

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'] )

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", 
                                        patience=5, restore_best_weights = True)

history = model.fit(train, validation_data = val, epochs=epochs, shuffle=True, verbose=1, 
                    callbacks = [earlystopping])

Epoch 1/50
 783/2447 [========>.....................] - ETA: 1:31:20 - loss: 0.3209 - accuracy: 0.9036

In [29]:
test = ImageDataGenerator(rescale=1./255).flow_from_directory("/home/adarsh/Jupyter_Notebook/ASL/test_ind/", 
                                                            target_size=(64, 64), class_mode=None)
pred = model.predict(test)
print(pred)
pred = np.argmax(pred, axis=1)
pred

Found 3 images belonging to 3 classes.
[[7.8346152e-15 3.9848319e-24 1.0000000e+00]
 [1.0000000e+00 2.8225668e-16 5.7566397e-13]
 [4.9033878e-25 1.0000000e+00 1.5106818e-12]]


array([2, 0, 1])

In [ ]:
model.save('vgg16asl.h5')